In [1]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


wdir = "../../resulting data/"


In [4]:
verses_features = pd.read_csv(wdir +  "verses_features.tsv", sep="\t")

In [5]:
verses_features.head()

,Unnamed: 0,bible_identifier,bible_book,bible_chapter,bible_book_chapter,relations,monogamy,poligamy,marriage,second-marriage,...,#per129,pla15,#pla74,pla69,org435,org297,org183,org102,#pla75,org22
0,b.GEN.001.001,b.GEN.001.001,NaN,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b.GEN.001.002,b.GEN.001.002,NaN,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b.GEN.001.003,b.GEN.001.003,NaN,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b.GEN.001.004,b.GEN.001.004,NaN,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b.GEN.001.005,b.GEN.001.005,NaN,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
verses_features.fillna(0, inplace=True)

In [7]:
verses_features.head()

,Unnamed: 0,bible_identifier,bible_book,bible_chapter,bible_book_chapter,relations,monogamy,poligamy,marriage,second-marriage,...,#per129,pla15,#pla74,pla69,org435,org297,org183,org102,#pla75,org22
0,b.GEN.001.001,b.GEN.001.001,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,b.GEN.001.002,b.GEN.001.002,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,b.GEN.001.003,b.GEN.001.003,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,b.GEN.001.004,b.GEN.001.004,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,b.GEN.001.005,b.GEN.001.005,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
verses_features[verses_features["coitus"] > 0 ]["bible_identifier"].values.tolist()

['b.GEN.004.001',
 'b.GEN.004.017',
 'b.GEN.004.025',
 'b.GEN.016.002',
 'b.GEN.016.003',
 'b.GEN.016.004',
 'b.GEN.020.004',
 'b.GEN.024.016',
 'b.GEN.024.067',
 'b.GEN.026.010',
 'b.GEN.029.023',
 'b.GEN.029.030',
 'b.GEN.030.003',
 'b.GEN.030.004',
 'b.GEN.030.016',
 'b.GEN.034.002',
 'b.GEN.034.007',
 'b.GEN.035.022',
 'b.GEN.038.002',
 'b.GEN.038.016',
 'b.GEN.038.018',
 'b.GEN.039.007',
 'b.GEN.039.010',
 'b.GEN.039.012',
 'b.GEN.039.014',
 'b.GEN.039.015',
 'b.GEN.039.017',
 'b.GEN.039.018',
 'b.GEN.049.004',
 'b.RUT.004.013',
 'b.MAT.001.025',
 'b.MAT.019.005',
 'b.MAT.019.006']

In [35]:
verses_features["number of 1st level quotations"].sum()

0.0

In [8]:
features_corrs = verses_features.corr()

In [31]:
features_corrs = features_corrs.reindex(features_corrs.max(1).sort_values().index, axis=1)

In [32]:
for column in features_corrs.columns.tolist():
    

,relations,#per2152,#per2203,#per2153,#per2154,#per2155,#per2324,#per1928,#per2156,#per2157,...,animals,plants,time,part-of-the-day,seasons,number of 1st level quotations,number of 2nd level quotations,number of 3rd level quotations,number of 4th level quotations,number of 5th level quotations
bible_book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bible_chapter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bible_book_chapter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
relations,1.000000,-0.000085,-0.000142,-0.000085,-0.000085,-0.000085,-0.000085,-0.000120,-0.000085,-0.000085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monogamy,-0.000209,-0.000079,-0.000131,-0.000079,-0.000079,-0.000079,-0.000079,-0.000112,-0.000079,-0.000079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
org297,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
org183,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
org102,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#pla75,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
features_corrs.dropna()

,relations,#per2152,#per2203,#per2153,#per2154,#per2155,#per2324,#per1928,#per2156,#per2157,...,animals,plants,time,part-of-the-day,seasons,number of 1st level quotations,number of 2nd level quotations,number of 3rd level quotations,number of 4th level quotations,number of 5th level quotations
relations,1.000000,-0.000085,-0.000142,-0.000085,-0.000085,-0.000085,-0.000085,-0.000120,-0.000085,-0.000085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monogamy,-0.000209,-0.000079,-0.000131,-0.000079,-0.000079,-0.000079,-0.000079,-0.000112,-0.000079,-0.000079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
poligamy,-0.000341,-0.000129,-0.000215,-0.000129,-0.000129,-0.000129,-0.000129,-0.000182,-0.000129,-0.000129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marriage,-0.000591,-0.000223,-0.000372,-0.000223,-0.000223,-0.000223,-0.000223,-0.000316,-0.000223,-0.000223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
second-marriage,-0.000120,-0.000046,-0.000076,-0.000046,-0.000046,-0.000046,-0.000046,-0.000064,-0.000046,-0.000046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
org297,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
org183,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
org102,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#pla75,-0.000085,-0.000032,-0.000054,-0.000032,-0.000032,-0.000032,-0.000032,-0.000046,-0.000032,-0.000032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
